In [ ]:
#Import necessary modules and packages
%matplotlib qt5
import numpy as np
import scipy.signal
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm_notebook
import time

# Defining necessary functions

In [ ]:
# MOUSE SELECT FUNCTION
def select_point(event, x, y, flags, params):
    global point_selected, points, point, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        points.append([[x,y]])#adding points to list
        point_selected = True

# SHOW FIRST FRAME OF VIDEO        
def get_frame(filename,index):
    counter=0
    video=cv2.VideoCapture(filename)
    while video.isOpened():
        rete,frame=video.read()
        if rete:
            if counter==index:
                return frame
            counter +=1
        else:
            break
    video.release()
    return None  

# FUNCTION TO COUNT FRAMES
def count_frames_manual(video):
    # initialize the total number of frames read
    total = 0
    # loop over the frames of the video
    video=cv2.VideoCapture(video)
    while True:
        # grab the current frame
        (grabbed, frame) = video.read()

        # check to see if we have reached the end of the video
        if not grabbed:
            break
        # increment the total number of frames read
        total += 1
    # return the total number of frames in the video file
    return total

# Rezise image to fit screen
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

 # Motion tracking  

In [ ]:
# Ask for user input
val = input("Mode shapes for which axis? x or y?: ")

if val == 'x':
    print('Place sensor from top to bottom on structure!')
    print('Press ESC when ready to track!')
else:
    print('Place sensor from left to right on structure!')
    print('Press ESC when ready to track!')


# PART ONE OF CODE WHERE WE PLACE SCENSORS

# Video to Track
file = "stochastic-response_25amp.mp4"

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)

points = []
point_selected = False 
frame = get_frame(file,0) 

while True:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if point_selected == True:
        cv2.circle(frame, point, 5, (0,0,255), 2)
       
    cv2.imshow("Frame",frame)   
    
    key = cv2.waitKey(1)
    if key == 27:
        break  
        
old_points = np.array(points, dtype=np.float32)    
cv2.waitKey(1)

# Nå kjøre tracker
# Video to track
cap = cv2.VideoCapture(file)

# Details about video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Count frames
total = count_frames_manual(file)

print("Total number of frames: %i" %total)
print("Number of fps: %i" %fps)

#CREATE OLD FRAME
_, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#LUKAS KANADE PARAMS
lk_params = dict(winSize = (30,30), # endre størrelse på søkerom 
                maxLevel = 8, # Number of pyramid levels
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)

#Empty variables to save tracking data
count = 0
x_values = np.zeros([len(old_points),total-1]) 
y_values = np.zeros([len(old_points),total-1])

pbar = tqdm_notebook(total = total)

while True:
    
    ret, frame = cap.read()
    
    if not ret:
        print('Video Finished')
        break 

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if point_selected == True:
        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_params)
        old_gray = gray_frame.copy()
        old_points = new_points
        
    
        for i in range(len(new_points)):
            x, y = new_points[i].ravel()
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
            
            # Lagrer alle punkter
            x_values[[i],[count]] = x
            y_values[[i],[count]] = y
      
    count = count+1   
        
    cv2.imshow("Frame", frame)
    pbar.update(1)

    key = cv2.waitKey(1)
    
    if key == 27:
        break
    
pbar.close()            
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

# Create time series and find sampling frequency

In [ ]:
T = total/fps
if val == 'x':
    Displacements = x_values
else:    
    Displacements = y_values
    
# Time signal from tracking
t = np.linspace(0, T, len(Displacements[0]))
dt = T/len(Displacements[0])

# Sample rate
Fs = 1/dt;
print("Sampling rate is: %f" %Fs)

# Plot displacement data

In [ ]:
#Plot Displacement 
if len(Displacements) == 1:
    plt.figure()
    plt.plot(t,Displacements[0])
    plt.grid()
    plt.xlabel('Time [s]')
    plt.ylabel('Response [pixels]')
    plt.title('Sensor 1')

else:
    # Plot in time domain
    fig, axs = plt.subplots(len(Displacements))
    plt.subplots_adjust(hspace=1)

    for i in range(len(Displacements)):
        axs[i].plot(t,Displacements[i])
        axs[i].set_title(f"sensor %d" % i)
        axs[i].set_xlabel('Time')
        axs[i].set_ylabel('Response')
        axs[i].grid()
plt.show()

# Estimate and plot spectrums using Welch' method

In [ ]:
#Estimate the spectrum using Welch' method
#Smoothing of spectrum
Nwelch=4 #Number of divisions of time series for averaging of spectrum
Nwindow=np.floor(np.max(np.shape(Displacements[0]))/Nwelch); # Length of window

#Empty variables
Spectrum = {}
peaks = {}
Value = {}
count = 0
diag = []
for n in range(len(Displacements)):   
    #Spectrums
    for h in range(len(Displacements)):
        f, Cross = scipy.signal.csd(Displacements[n], Displacements[h],Fs,nperseg=Nwindow)
        Spectrum[count] = Cross 
        if n == h:
            diag.append(count)   
        count = count + 1   

#Plot PSD in frequency domain using for loop    
if len(Displacements) == 1:
    plt.figure()
    plt.plot(f,np.real(Spectrum[0]))
    plt.grid()
    plt.xlabel('f [Hz]')
    plt.ylabel('PSD')
    plt.title('Sensor 1')

else:
    fig, axs = plt.subplots(len(Displacements),len(Displacements))
    plt.subplots_adjust(hspace=1)
    count = 0
    for b in range(len(Displacements)):
        for k in range(len(Displacements)):
            #axs[b,k].semilogy(f, np.real(Spectrum[count]))
            axs[b,k].plot(f,np.real(Spectrum[count]))
            count=count+1
            axs[b,k].set_title(f"sensor %d %d" %(b,k))
            #axs[b,k].set_xlabel('f [Hz]')
            #axs[b,k].set_ylabel('PSD')
            #axs[b,k].set_yscale('log')
            axs[b,k].set_xlim([0,33])
            axs[b,k].grid()
plt.show()

plt.figure()
plt.xlim([0,33])
plt.semilogy(f, np.real(Spectrum[0]))
#plt.plot(f,np.real(Spectrum[0]))
selected = plt.ginput(n=-1, timeout = 0)

# Calculate and plot mode shapes

In [ ]:
Scale = float(input("How much would you like to amplify the Mode shapes?: "))

# PLot orignial sensors placement!
plt.figure()
plt.xlim([0,width])
plt.grid()
diff = []

x_orig = []
y_orig = []
for i in range(len(points)):
    if val =='x':
        x_orig.append(points[i][0][0])
        y_orig.append(-points[i][0][1])
        plt.plot(points[i][0][0],-points[i][0][1],'x', linestyle="--")
        diff.append(max(Displacements[i])-min(Displacements[i]))
        max_value = max(diff)
        max_index = diff.index(max_value)
    else: 
        x_orig.append(points[i][0][0])
        y_orig.append(points[i][0][1])
        plt.plot(points[i][0][0],points[i][0][1],'x', linestyle="--")
        diff.append(max(Displacements[i])-min(Displacements[i]))
        max_value = max(diff)
        max_index = diff.index(max_value)

    
#Find frequencies from chosen peaks
Frequencies = []     
for i in range(len(selected)):
    Frequencies.append(selected[i][0])

#Find index of nearest value
index = []
for k in range(len(Frequencies)):
    difference_array = np.absolute(f-Frequencies[k])
    index.append(difference_array.argmin())
Variable = index

#Find mode shapes
ModeShapes = np.zeros([len(Frequencies),len(points)]) # Create empty mode shapes

for k in range(len(Frequencies)):
    count = max_index
    for m in range(len(points)-1):
        ModeShapes[k][m] = np.sqrt(np.real(Spectrum[diag[m]][Variable[k]])/np.real(Spectrum[diag[max_index]][Variable[k]]))*np.sign(np.real(Spectrum[count][Variable[k]]))
        count = count+len(points)  

#Scale mode shapes
ModeShapes = ModeShapes*Scale

#Plot mode shapes
for n in range(len(Frequencies)):
    x_points=[]
    y_points=[]
    plt.figure()
    #plt.xlim([0,width])
    #plt.ylim([0,-height])
    plt.grid()
    plt.title("Modeshape frequency. %f Hz." %Frequencies[n] + "  Scaled by %i"  %Scale )
    for p in range(len(points)):
        if val == 'x':
            plt.xlim([0,width])
            x_points.append(ModeShapes[n][p]+points[p][0][0])
            y_points.append(-points[p][0][1])
        else:
            plt.xlim([0,width])
            plt.ylim([0,height])
            y_points.append(ModeShapes[n][p]+points[p][0][1])
            x_points.append(points[p][0][0])
    plt.plot(x_points, y_points, 'x', linestyle="--", label='Mode shape')
    plt.plot(x_orig, y_orig, 'o', linestyle="-", label='Original Shape')
    plt.legend()